In [1]:
!pip install osmnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 92 kB 142 kB/s 
     |████████████████████████████████| 62 kB 605 kB/s 
     |████████████████████████████████| 1.1 MB 44.1 MB/s 
     |████████████████████████████████| 1.0 MB 32.7 MB/s 
     |████████████████████████████████| 9.4 MB 26.2 MB/s 
     |████████████████████████████████| 7.8 MB 25.7 MB/s 
     |████████████████████████████████| 17.1 MB 2.4 MB/s 
     |████████████████████████████████| 12.2 MB 14.2 MB/s 
     |████████████████████████████████| 16.6 MB 13.2 MB/s 
     |████████████████████████████████| 965 kB 20.7 MB/s 
     |████████████████████████████████| 295 kB 56.0 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      

In [ ]:
#!python3 complex_model.py

/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Iterations: 100
Proportion Bad: 0.0
Mean Iter. Good: 40540.5
Iterations: 100
Proportion Bad: 0.1
Mean Iter. Good: 40268.17853231106
Iterations: 100
Proportion Bad: 0.2
Mean Iter. Good: 40834.65260545906
Iterations: 62
Proportion Bad: 0.30000000000000004
Mean Iter. Good: 40342.81742738589
Iterations: 100
Proportion Bad: 0.4
Mean Iter. Good: 40953.40909090909
Iterations: 38
Proportion Bad: 0.5
Mean Iter. Good: 42142.48093220339
Iterations: 100
Proportion Bad: 0.6000000000000001
Mean Iter. Good: 39269.34782608696

^C


In [11]:
from google.colab import drive
drive.mount('/content/drive/')
import os

# TODO: Fill in the Google Drive path where you uploaded
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive/
['Copy of Welcome To Colaboratory', 'Untitled0.ipynb', 'MOCS_final_rand_init.ipynb']


In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import random
from gen_complex_net import gen_net
from gen_complex_net import gen_data

In [3]:
import copy


In [4]:
def generate_drivers(num_drivers: int, bad_driver_prop: float, states: list) -> list:
	"""
	generate drivers generates an array of randomly chosen good and bad drivers

	Params:
	num_drivers: total number of drivers
	bad_driver_prop: proportion of bad drivers on the network
	states: state of drivers

	Returns:
	list of drivers and their state
	"""
	drivers = [{key : {'State' : random.choices(states, [1-bad_driver_prop, bad_driver_prop])[0],
										 'Iterations' : 0}} for key in np.arange(1,num_drivers+1, 1)]
	return drivers

In [37]:
# Setup for testing
num_drivers = 1000
prop_bad = 0.1
driver_list = generate_drivers(num_drivers, prop_bad, states = ["good","bad"])
net = gen_net(edges=gen_data(), node_vals=["u", "v", "length"])


In [35]:
def run_model_rand_init(driver_list: list, net: nx.Graph, prob_wrong_turn: float):
  curr_nodes = random.choices(list(net.nodes), k=len(driver_list))
  end_nodes = random.choices(list(net.nodes), k=len(driver_list))
  for i in range(len(driver_list)):
    path = (nx.shortest_path(net,
                                  source=curr_nodes[i],
                                  target=end_nodes[i],
                                  method='dijkstra',
                                  weight="length"))
    driver_list[i]['path'] = path
    driver_list[i]['path_length'] = len(path)
    driver_list[i]['end'] = end_nodes[i]
    driver_list[i]['complete'] = 'incomplete'
    curr_node = copy.deepcopy(curr_nodes[i])
    net.nodes[curr_node]["Queue"].append(driver_list[i])

  incomplete = 1
  iter = 0
  good_finished = 0
  bad_finished = 0
  comp_good = dict()
  comp_bad = dict()
  while incomplete != 0:
    incomplete = 0
    pop_nodes = []
    for driver in driver_list:
      if driver['complete'] == 'incomplete':
        driver_num = list(driver.keys())[0]
        node = driver['path'][0]
        first_in_queue = list(net.nodes[node]['Queue'][0].keys())[0]
        if first_in_queue != driver_num:
          pass
        else:
          if node == driver['end']:
            #popped = net.nodes[node]['Queue'].pop(0)
            driver['complete'] = 'complete'
            if driver[driver_num]['State'] == 'good':
              good_finished += 1
            else:
              bad_finished += 1
          else:
            wrong_turn = False
            if driver[driver_num]['State'] == 'bad':
              turn_choice = random.choices(["on_path", "off_path"], [1-prob_wrong_turn, prob_wrong_turn])[0]
              if turn_choice == "off_path":
                wrong_turn = True
            if wrong_turn:
              #popped = net.nodes[node]['Queue'].pop(0)
              next_node = random.choice([n for n in net.neighbors(node)])
              driver['path'] = nx.shortest_path(net, source = next_node, target = driver['end'], method = 'dijkstra', weight = 'length')
              net.nodes[next_node]['Queue'].append(driver)
              pass
            else:
              #popped = net.nodes[node]['Queue'].pop(0)
              next_node = driver['path'][1]
              driver['path'] = driver['path'][1:]
              net.nodes[next_node]['Queue'].append(driver)
          # Mark node for popping
          pop_nodes.append(node)
      if driver['complete'] == 'incomplete':
          incomplete += 1
          driver[driver_num]['Iterations'] += 1
    iter += 1
    if good_finished != 0:
      comp_good[good_finished] = copy.deepcopy(iter)
    if bad_finished != 0:
      comp_bad[bad_finished] = copy.deepcopy(iter)
    # Pop nodes
    for n in pop_nodes:
      popped = net.nodes[n]['Queue'].pop(0)


  iteration_list = [list(driver.values())[0]['Iterations'] / driver['path_length'] for driver in driver_list]
  final_good = pd.DataFrame.from_dict(comp_good, orient = 'index')
  final_bad = pd.DataFrame.from_dict(comp_bad, orient = 'index')
  path_sum = 0
  for driver in driver_list:
    path_sum += driver['path_length']
  av_path_length = path_sum / len(driver_list)

  return iteration_list, final_good, final_bad, av_path_length
  

In [38]:
it_list, fg, fb, apl = run_model_rand_init(driver_list, net, 0.01)

In [39]:
apl

74.467

In [40]:
def run_and_plot(num_drivers: int, scale: int):
	prop_bad = scale * 0.1
	driver_list = generate_drivers(num_drivers, prop_bad, states = ["good","bad"])
	net = gen_net(edges=gen_data(), node_vals=["u", "v", "length"])
	total, good, bad, apl = run_model_rand_init(driver_list=driver_list,
															net=net,
															prob_wrong_turn=0.01)
	good.to_csv(GOOGLE_DRIVE_PATH + '/gooddf{}'.format(scale))
	bad.to_csv(GOOGLE_DRIVE_PATH + '/baddf{}'.format(scale))
	print('Proportion Bad {}'.format(prop_bad))
	print('Iterations/Path_len: {}'.format(max(total)))
	print('Mean Iter. Good: {}'.format(good.mean(0)[0]))
	print('Average Path Length {}'.format(apl), '\n')


In [41]:
def run_and_plot_vary_num_drivers(num_drivers: int, scale: int):
  prop_bad = scale * 0.1
  driver_list = generate_drivers(num_drivers, prop_bad, states = ["good","bad"])
  net = gen_net(edges=gen_data(), node_vals=["u", "v", "length"])
  total, good, bad, apl = run_model_rand_init(driver_list=driver_list,
                              net=net,
                              prob_wrong_turn=0.01)
  good.to_csv(GOOGLE_DRIVE_PATH + '/gooddf{}'.format(num_drivers))
  bad.to_csv(GOOGLE_DRIVE_PATH + '/baddf{}'.format(num_drivers))
  print('Proportion Bad {}'.format(prop_bad))
  print('Num Drivers {}'.format(num_drivers))
  print('Iterations/Path Length: {}'.format(np.mean(total)))
  print('Mean Iter. Good: {}'.format(good.mean(0)[0]))
  print('Average Path Length {}'.format(apl), '\n')

In [47]:
def run_and_plot_vary_prob_wrong_turn(num_drivers: int, scale: int, prob_wrong_turn: float):
  prop_bad = scale * 0.1
  driver_list = generate_drivers(num_drivers, prop_bad, states = ["good","bad"])
  net = gen_net(edges=gen_data(), node_vals=["u", "v", "length"])
  total, good, bad, apl = run_model_rand_init(driver_list=driver_list,
                              net=net,
                              prob_wrong_turn=prob_wrong_turn)
  good.to_csv(GOOGLE_DRIVE_PATH + '/gooddf_prob{}'.format(prob_wrong_turn*100))
  bad.to_csv(GOOGLE_DRIVE_PATH + '/baddf_prob{}'.format(prob_wrong_turn*100))
  print('Proportion Bad {}'.format(prop_bad))
  print('Num Drivers {}'.format(num_drivers))
  print('Iterations: {}'.format(max(total)))
  print('Mean Iter. Good: {}'.format(good.mean(0)[0]))
  print('Prob Wrong Turn {}'.format(prob_wrong_turn))
  print('Average Path Length {}'.format(apl), '\n')

In [43]:
# Get plots

num_drivers = 1000
sig_digits = 10

for i in range(sig_digits):
  run_and_plot(num_drivers, i)


Proportion Bad 0.0
Iterations/Path_len: 2.7205882352941178
Mean Iter. Good: 127.94214876033058
Average Path Length 70.301 

Proportion Bad 0.1
Iterations/Path_len: 3.180327868852459
Mean Iter. Good: 134.78571428571428
Average Path Length 73.024 

Proportion Bad 0.2
Iterations/Path_len: 2.823529411764706
Mean Iter. Good: 130.64197530864197
Average Path Length 74.048 

Proportion Bad 0.30000000000000004
Iterations/Path_len: 2.769230769230769
Mean Iter. Good: 128.2900432900433
Average Path Length 72.48 

Proportion Bad 0.4
Iterations/Path_len: 2.6785714285714284
Mean Iter. Good: 125.16666666666667
Average Path Length 73.474 

Proportion Bad 0.5
Iterations/Path_len: 3.255813953488372
Mean Iter. Good: 122.05687203791469
Average Path Length 72.71 

Proportion Bad 0.6000000000000001
Iterations/Path_len: 3.263157894736842
Mean Iter. Good: 118.44919786096257
Average Path Length 73.924 

Proportion Bad 0.7000000000000001
Iterations/Path_len: 3.169811320754717
Mean Iter. Good: 127.26506024096386


In [44]:
# Get plots varying num_drivers
min_num_drivers = 1000
max_num_drivers = 10000
step = 1000

for i in np.arange(start = min_num_drivers, stop = max_num_drivers+1, step = step):
  run_and_plot_vary_num_drivers(i, 1) # Prop bad drivers 0.1

Proportion Bad 0.1
Num Drivers 1000
Iterations/Path Length: 1.4579267505562352
Mean Iter. Good: 127.00418410041841
Average Path Length 73.344 

Proportion Bad 0.1
Num Drivers 2000
Iterations/Path Length: 2.373311991324882
Mean Iter. Good: 245.50108459869847
Average Path Length 73.8325 

Proportion Bad 0.1
Num Drivers 3000
Iterations/Path Length: 3.328734417354354
Mean Iter. Good: 321.88178913738017
Average Path Length 73.478 

Proportion Bad 0.1
Num Drivers 4000
Iterations/Path Length: 4.248795932697623
Mean Iter. Good: 429.21109770808204
Average Path Length 72.51425 

Proportion Bad 0.1
Num Drivers 5000
Iterations/Path Length: 5.178207928783019
Mean Iter. Good: 501.28309572301424
Average Path Length 72.9288 

Proportion Bad 0.1
Num Drivers 6000
Iterations/Path Length: 6.2208507004845375
Mean Iter. Good: 618.6270764119602
Average Path Length 72.98366666666666 

Proportion Bad 0.1
Num Drivers 7000
Iterations/Path Length: 6.95152073842206
Mean Iter. Good: 696.6664212076583
Average Path L

In [49]:
# Get plots varying wrong turn percentage
num_drivers = 5000
min_prob = .01
max_prob = .10
step = .01

for i in np.arange(start = min_prob, stop = max_prob+1, step = step):
  run_and_plot_vary_prob_wrong_turn(num_drivers, 1, i) # Prop bad drivers 0.1
# Cut off runtime after prob 0.24

Proportion Bad 0.1
Num Drivers 5000
Iterations: 17.945945945945947
Mean Iter. Good: 512.1133603238867
Prob Wrong Turn 0.01
Average Path Length 72.5306 

Proportion Bad 0.1
Num Drivers 5000
Iterations: 17.571428571428573
Mean Iter. Good: 528.3307392996109
Prob Wrong Turn 0.02
Average Path Length 73.134 

Proportion Bad 0.1
Num Drivers 5000
Iterations: 21.636363636363637
Mean Iter. Good: 523.7309197651664
Prob Wrong Turn 0.03
Average Path Length 73.5536 

Proportion Bad 0.1
Num Drivers 5000
Iterations: 15.829268292682928
Mean Iter. Good: 526.9484435797665
Prob Wrong Turn 0.04
Average Path Length 72.8882 

Proportion Bad 0.1
Num Drivers 5000
Iterations: 18.21212121212121
Mean Iter. Good: 539.3514804202483
Prob Wrong Turn 0.05
Average Path Length 73.3774 

Proportion Bad 0.1
Num Drivers 5000
Iterations: 19.186046511627907
Mean Iter. Good: 539.1296472831268
Prob Wrong Turn 0.060000000000000005
Average Path Length 72.9494 

Proportion Bad 0.1
Num Drivers 5000
Iterations: 20.09090909090909
Me

KeyboardInterrupt: ignored